In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys; sys.path.insert(0, '..')

import src.pipelines.messenger_pipeline as messenger_pipeline
import src.utils.database as db
from datetime import datetime

from bson import ObjectId


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

## Themes extruction v1

Переписываем текст из громоздкого JSON в простой формат, где одна строка = одна реплика:


In [5]:
chunks_collection = db.get_collection('chunks')
chunks = chunks_collection.find({"document_id": ObjectId("676efafcb7cb279c1e2dd663")}).to_list()

In [ ]:
messenger_pipeline.to_plain_text([x.get('_id') for x in chunks.to_list()])

Прогоняем все чанки через LLM, чтобы извлечь темы и ключевые слова

In [47]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [ ]:
chunks = chunks_collection.find({
  "document_id": ObjectId("676efafcb7cb279c1e2dd663"),
  "processing_log": {"$not": {"$regex": "theme_extraction"}}
}).to_list()

In [87]:
theme_extraction_prompt= ChatPromptTemplate.from_template(
  """# Задача 
Внимательно изучи приведенный ниже текст переписки и выделить из него ВСЕ темы, которые затрагиваются в диалоге, без исключений. 

# Этапы выполнения задачи:
1. Проанализируй структуру и хронологию диалога, определи скрытые контексты и взаимосвязи между репликами (например, упоминания событий, людей или задач);

2. Определи ключевые этапы диалога и выдели общий список затрагиваемых вопросов. 

3. Проанализируй каждый этап и каждый затрагиваемый вопрос отдельно, чтобы ВСЕ БЕЗ ИСКЛЮЧЕНИЯ, затрагиваемые или косвенно связанные с разговором темы.


Требования:  
1. Выделяй только конкретные темы, избегая общих и размытах. Формулируй темы кратко и конкретно. Тема должна быть сформулирована так, чтобы по ней было легко найти диалог среди других.
 
Пример хороших тем:
[
"Перечень работ и запчастей от страховой компании",
"Генерация изображений для Aito нейросетями",
"Сравнение приложений для  ведения заметок",
"Недовольство финансовым отчетом компании",
"Выдача доступа к Google Диску новому сотруднику Мише", 
]

Пример плохих тем, не допускай такого в выдаче:
[
"Предоставление услуг",
"Сотрудничество",
"Обсуждение рендеров", 
"Обсуждение формата изображений"
]

2. Тщательно проверяй текст, чтобы ничего не упустить и выделить ВСЕ темы, даже если они упоминаются мельком или косвенно.  

3. Ни в коем случае не пропускай мелкие, но конкретные вопросы, которые обсуждались. Обращай внимание на детали. Это очень важно.

4. Не придумывай факты, опирайся только на информацию из переписки и контекст.

5. Обязательно избегай повторения перечислений одной темы разными словами.

6. Укажи столько тем, сколько нужно чтобы полностью передать содержание диалога без потери смысла. Чем большее деталей, тем лучше. 

Ответ должен быть списком тем в формате JSON. 


###########
# Input
{dialog}

###########
Перед тем как отправлять ответ, проверь ещё раз, не пропустил ли ты какую-то тему.  Не указывай в списке повторяющиеся темы и перечисления одной темы разными словами. 

# Output
"""
)

In [88]:
class ThemesExtractionResponse(BaseModel):
  themes: list[str] = Field(description="Выделенные из разговора темы")

In [89]:
llm = ChatOpenAI(model="gpt-4o-mini").with_structured_output(ThemesExtractionResponse)

In [90]:
log_str = '{timestamp}|theme_extraction|v1.0.0'.format(timestamp=datetime.now())
for chunk in chunks: 
  prompt = theme_extraction_prompt.invoke({"dialog": chunk.get('plain_text') })
  response = llm.invoke(prompt)

  print(response.model_dump().get('themes'))

  chunks_collection.update_one(
    {"_id": chunk["_id"]}, 
    {
      "$set": {
        "themes": response.model_dump().get('themes'), 
        "processing_log": [log_str] + chunk["processing_log"]
      },
    }
  )

['Внепланный соцопрос о дизайне стола', 'Размеры стола и уровень громкости', 'Контактные данные и социальные сети для worki.space', 'Типы столов: офисный, регулируемый, ученический', 'Отправка рендеров интерьера дизайнеру', 'Ссылки на регулируемые столы на Ozon', 'Проблемы с отзывами: стоимость, необходимость живых видео', 'Рекомендации по накрутке отзывов', 'Стоимость и условия выполнения заказов', 'Необходимость размещения SKU и работы с объявлениями', 'Проблемы с отзывами и оформлением профиля', 'Оптимизация семантики для объявлений', 'Автообзвон по базе маркетолога и владельцев грузовиков', 'Обсуждение цены за фотографию в зависимости от объема заказа', 'Передача документов за август']
['Коды подтверждения для авторизации на my.telegram.org', 'Безопасность кодов авторизации в Telegram', 'Игнорирование незапрашиваемых сообщений о кодах', 'Использование кода для входа в Telegram']
['Оплата заказа через P2P.Kassa', 'Поддержка и работа службы поддержки', 'Обновления в приложении Wantto

## Retreive test

Векторизуем все теги (и на всякий случай plain_text тоже), засовываем в БД и тестим!

In [6]:
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

In [8]:
chunks = chunks_collection.find({
  "document_id": ObjectId("676efafcb7cb279c1e2dd663"),
  # "processing_log": {"$not": {"$regex": "theme_extraction"}}
}).to_list()
chunks

[{'_id': ObjectId('676efafcb7cb279c1e2dd664'),
  'chunk_id': '7066936566_0',
  'document_id': ObjectId('676efafcb7cb279c1e2dd663'),
  'chunk_index': 0,
  'processing_log': ['2024-12-27 22:37:39.253635|theme_extraction|v1.0.0',
   '2024-12-27 22:07:40.247719|to_plain_text|v1.0.1'],
  'chunk_metadata': {'dialogId': 7066936566,
   'name': 'Worki Team',
   'dialogType': 'User',
   'timestampFrom': '2024-12-16T23:39:29+00:00',
   'timestampTo': '2024-07-15T10:27:22+00:00'},
  'original_data': '[{"id": 2702, "date": "2024-12-16T23:39:29+00:00", "sender_id": 7066936566, "text": "Hello, myself!", "reply_to": null}, {"id": 2701, "date": "2024-12-16T23:30:58+00:00", "sender_id": 7066936566, "text": "Hello, myself!", "reply_to": null}, {"id": 2700, "date": "2024-12-16T23:29:43+00:00", "sender_id": 7066936566, "text": "Hello, myself!", "reply_to": null}, {"id": 2586, "date": "2024-10-09T11:03:31+00:00", "sender_id": 7066936566, "text": "Документы за август в с7", "reply_to": null}, {"id": 2462, "d

In [18]:
docs = []
for chunk in chunks: 
  for theme in chunk.get('themes'):
    docs.append(Document(page_content=theme, metadata={
      "chunk_id": str(chunk['_id'])
    }))
  docs.append(Document(page_content=chunk.get('plain_text'), metadata={
    "chunk_id": str(chunk['_id'])
  }))
  docs.append(Document(page_content=' '.join(chunk.get('themes')), metadata={
    "chunk_id": str(chunk['_id'])
  }))
docs

[Document(metadata={'chunk_id': '676efafcb7cb279c1e2dd664'}, page_content='Внепланный соцопрос о дизайне стола'),
 Document(metadata={'chunk_id': '676efafcb7cb279c1e2dd664'}, page_content='Размеры стола и уровень громкости'),
 Document(metadata={'chunk_id': '676efafcb7cb279c1e2dd664'}, page_content='Контактные данные и социальные сети для worki.space'),
 Document(metadata={'chunk_id': '676efafcb7cb279c1e2dd664'}, page_content='Типы столов: офисный, регулируемый, ученический'),
 Document(metadata={'chunk_id': '676efafcb7cb279c1e2dd664'}, page_content='Отправка рендеров интерьера дизайнеру'),
 Document(metadata={'chunk_id': '676efafcb7cb279c1e2dd664'}, page_content='Ссылки на регулируемые столы на Ozon'),
 Document(metadata={'chunk_id': '676efafcb7cb279c1e2dd664'}, page_content='Проблемы с отзывами: стоимость, необходимость живых видео'),
 Document(metadata={'chunk_id': '676efafcb7cb279c1e2dd664'}, page_content='Рекомендации по накрутке отзывов'),
 Document(metadata={'chunk_id': '676efaf

In [19]:
# vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())
vectorstore = Chroma(
    collection_name="dialogs",
    embedding_function=OpenAIEmbeddings(),
    persist_directory="../app_data"  # Directory to save data
)
vectorstore.add_documents(docs)

['2a6d8660-3ba3-4bd8-9364-07ec8ff6968b',
 'c35e314f-82f3-4c3c-916a-710bc1278c92',
 '5a6a89c7-daf6-4b18-b3a9-44abeaa7c4a7',
 '59782a5d-7035-415b-afb6-7ea96c036cdf',
 'aaa06cdc-2509-472d-a16f-9c12023bd1f5',
 '8263c2b5-9c23-4160-8f1b-06e09e1d0da7',
 '22b3286a-4295-4061-ae36-620f5ac8a1ff',
 'b7cd3236-8b63-44ba-ba1f-e5374290eed9',
 'a9e8130c-58f7-4c5f-9036-f6e790641f9f',
 'f23fcdc0-3d10-4c8b-822d-76033f159772',
 '745a794a-cad3-41c2-b5f2-9477ab9c1781',
 '2c7c3509-e417-465b-aa21-50f37b517c64',
 'bf39ab8f-5e3a-4e58-baf5-d59a52c3a3db',
 'ed4f1dcb-76bc-4b87-8a38-70b63badb873',
 '2a9d50fe-d432-4128-8a3d-aadbd4ff1077',
 '073c7cbd-3886-43e5-acdd-477f0c3131ec',
 'af1c2e0e-8e29-4006-bf18-e25c582e3cd5',
 '69be1a1b-37ae-4e84-bc04-18d4ddc3370c',
 '99ecb28e-31b2-447e-a0b5-76fae2e95aa3',
 'e12a1eb1-c201-4ad3-a492-071d325515d9',
 'a41905c4-1d89-438e-9f29-2ec2508658e7',
 '822c897a-ab52-4261-874b-d2277b20d003',
 '0e138e0c-15c1-41bf-a470-b7e6521bf5a8',
 'eaf5c2f1-3c8f-4509-88ff-c6d07eb9640e',
 'd9e83135-930f-

In [12]:
results = vectorstore.similarity_search_with_score(
  "дизайнер, оплата, AI картинки, стоимость услуг, совместная работа, обсуждение проекта"
)
print(results)
for res, score in results:
  print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

[]


🚀
Выглядит очень прилично, явно лучше чем в прошлый раз, находи примерно те куски, которые я были нужны. 

Непоятно только, сколько выбирать кусков. Для анализа переписки нам нужен не самый релеватный кусок, а все существенные. А они могут в контекст не влезть... возможно придется уменьшать размер чанков

## Собираем простой QА чтобы протестить
Задача решается в 2 этапа:
1. Просим бота сгенерить поисковый запрос

2. Потом классический RAG QA

### 1. Генерим поисковый запрос слова:

In [22]:
gen_search_query_prompt = ChatPromptTemplate.from_template(
  """Входной вопрос: {question}

# Задача 
Напиши поисковый запрос, который извлечёт релевантные фрагменты из архива переписок. Помни: ответ на вопрос пользователя не указан явно в переписках. Поэтому поисковый запрос должен быть составлен так, чтобы извлечь ключевые фрагменты, связанные с темой разговора, вопросами или проблемами, обсуждаемыми в переписке, а не просто переформулировать исходный вопрос.

#Требования
1. Поисковый запрос должен быть максимально тематическим и включать ключевые слова, связанные с контекстом вопроса.
2. Используй используй понятные слова, которые помогут в semantic search
3. Старайся учитывать возможные варианты формулировок и синонимы, которые могли быть использованы в архиве.

#Примеры
Вопрос: в каком ресторане я отмечал день рождения в прошлом году?
Поисковый запрос: подготовка к деню рождения, выбор ресторана, организация праздника

Вопрос: какого цвета забор я установил на даче? 
Поисковый запрос: установка забора,  благоустройство дачного участка, ремонт

В ответ пришли только короткий поисковый запрос и ничего больше"""
)

NameError: name 'ChatPromptTemplate' is not defined

In [316]:
promptGetLLM = ChatOpenAI(model="gpt-4o-mini")

In [20]:
userQuestion = 'во сколько я встречался с команией которая делала 3д конфигураторы для моего мебельного проекта? летом было'

In [21]:
prompt = gen_search_query_prompt.invoke({'question': userQuestion })
searchQuery = promptGetLLM.invoke(prompt)
searchQuery.content

NameError: name 'gen_search_query_prompt' is not defined

### 2. Делаем RAG

In [324]:
retreivedData = vectorstore.similarity_search_with_score(
  searchQuery.content
)
retreivedData

[]

In [323]:
retreivedChunks = chunks_collection.find({"_id": {"$in": list(set([ObjectId(res.metadata.get('chunk_id')) for res, score in retreivedData]))}}).to_list()
context = '\n\n'.join([x.get('plain_text') for x in retreivedChunks])
context

''

In [321]:
qa_query_prompt = ChatPromptTemplate.from_template(
  """Входной вопрос пользователя: {question}

# Фрагменты перписки для контекста
{context}

Ты - полезный ассистент, который умеет анализировать информацию и отвечать на вопросы пользователя. Используя представленные фрагменты переписок пользователя, ответь на вопрос который он задал. Отвечай кратко и по существу. В ответ пришли только ответ на вопрос и ничего больше.
"""
)

In [322]:
prompt = qa_query_prompt.invoke({'question': userQuestion, 'context': context })
searchQuery = promptGetLLM.invoke(prompt)
print(searchQuery.content)

Вы встречались с компанией, которая делала 3D конфигураторы для вашего мебельного проекта, летом. Точное время не указано.
